**Q1: Triple Step** A child is running up a staircase with n steps and can hop either 1 step, 2 steps, or 3 steps at a time.  Implement a method to count how many possible ways the child can run up the stairs.

**Recursive, top-down solution** The simplest approach, conceptually, might be to start from the top of the staircase and work our way, recursively, back to the bottom.

The last hop to get to the top will be either 1, 2, or 3 steps.  So, the number of ways to get to the top will be the sum of the number of ways to get to the last step before the top, plus the number of ways to get to the step before that, plus the number of ways to get to the step before that.

In [1]:
def steps_rec_back(n):
    """Recursive top-down solution. (brute force)"""
    if n < 0:
        return 0
    if n == 0:
        return 1
    return steps_rec_back(n-1) + steps_rec_back(n-2) + steps_rec_back(n-3)

In [2]:
steps_rec_back(15)

5768

In [3]:
from functools import lru_cache

@lru_cache()
def steps_rec_back_memo(n):
    """Recursive top-down solution, with memoization."""
    if n < 0:
        return 0
    if n == 0:
        return 1
    return steps_rec_back_memo(n-1) + steps_rec_back_memo(n-2) + steps_rec_back_memo(n-3)

In [4]:
steps_rec_back_memo(15)

5768

In [5]:
def steps_iter_fwd(n):
    """Iterative bottom-up solution."""
    nsteps = [1] + [0] * n
    for i in range(n):
        nsteps[i+1] += nsteps[i]
        if i < n - 1:
            nsteps[i+2] += nsteps[i]
        if i < n - 2:
            nsteps[i+3] += nsteps[i]
    return nsteps[-1]

In [6]:
steps_iter_fwd(15)

5768

In [7]:
def steps_rec_fwd(n, s=1, s1=0, s2=0, s3=0):
    """Recursive bottom-up solution."""
    if n:
        return steps_rec_fwd(n-1, s+s1, s+s2, s+s3)
    return s

In [8]:
steps_rec_fwd(15)

5768

**Compare the four solutions**

In [9]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [10]:
def plot_timings(times, funcs):
    """Plot the evaluation times of the given functions as function of the input parameter, n."""
    t = [v[0] for v in times]
    names = [fn.__name__ for fn in funcs]
    
    fig = figure(title="Function Scaling",
                 x_axis_label='N',
                 y_axis_label='time (sec)')

    # add a line renderer with legend and line thickness
    for i, name in enumerate(names):
        fig.line(t, [v[i+1] for v in times], legend=name, line_width=2)

    # show the results
    show(fig)

In [11]:
import timeit

COL_FMT = '{:5d} steps ({:.3f} sec)'
HDR_FMT = '{:^24s}'

def time_fns(funcs, maxn=20, maxt=5, neval=20000, plot=False):
    """Time the given functions for n from 0 to maxn.
    If any evaluation takes more than maxt seconds, stop there.
    Each function is evaluated neval times using timeit().
    """
    print(" N" + " ".join([HDR_FMT.format(fn.__name__) for fn in funcs]))
    result = []
    for n in range(maxn):
        cols = ['{:2d}'.format(n)]
        times = []
        for fn in funcs:
            val = fn(n)
            dt = timeit.timeit('fn(n)', number=neval, globals=locals())
            times.append(dt)
            cols.append(COL_FMT.format(val, dt))
        result.append([n] + times)
        if max(times) > maxt:
            break
        print(" ".join(cols))
        
    if plot:
        plot_timings(result, funcs)

In [12]:
def steps_rec_back_memo_fresh(n):
    steps_rec_back_memo.cache_clear()
    return steps_rec_back_memo(n)

fns = [steps_rec_back, steps_rec_back_memo_fresh, steps_iter_fwd, steps_rec_fwd]

times = time_fns(fns, maxn=20, maxt=1, plot=True)

 N     steps_rec_back      steps_rec_back_memo_fresh      steps_iter_fwd           steps_rec_fwd      
 0     1 steps (0.006 sec)     1 steps (0.025 sec)     1 steps (0.022 sec)     1 steps (0.005 sec)
 1     1 steps (0.016 sec)     1 steps (0.051 sec)     1 steps (0.030 sec)     1 steps (0.009 sec)
 2     2 steps (0.028 sec)     2 steps (0.060 sec)     2 steps (0.037 sec)     2 steps (0.014 sec)
 3     4 steps (0.051 sec)     4 steps (0.073 sec)     4 steps (0.044 sec)     4 steps (0.020 sec)
 4     7 steps (0.100 sec)     7 steps (0.090 sec)     7 steps (0.059 sec)     7 steps (0.023 sec)
 5    13 steps (0.176 sec)    13 steps (0.110 sec)    13 steps (0.069 sec)    13 steps (0.034 sec)
 6    24 steps (0.337 sec)    24 steps (0.120 sec)    24 steps (0.081 sec)    24 steps (0.036 sec)
 7    44 steps (0.607 sec)    44 steps (0.143 sec)    44 steps (0.093 sec)    44 steps (0.040 sec)
